In [6]:
# Necessary library installation
!pip install openai==0.28
!pip install pandas

import openai
import os
import pandas as pd

# Openai API key setting
openai.api_key = 'sk'

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_text_from_file(file_path):
    """텍스트 파일을 읽어서 내용을 반환합니다."""
    with open(file_path, 'r') as file:
        content = file.read()
    return content

def split_into_sentences(text):
    """텍스트를 문장 단위로 분리합니다."""
    import nltk
    nltk.download('punkt')
    from nltk.tokenize import sent_tokenize
    sentences = sent_tokenize(text)
    return sentences

def evaluate_esg(sentence):
    """업데이트된 OpenAI API를 사용하여 문장의 ESG 친화도를 평가합니다."""
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # 혹은 사용 가능한 최신 모델
        messages=[
            {"role": "system", "content": "당신은 ESG 평가 전문가입니다. 먼저 ESG에 대한 설명을 제공해드릴테니 평가의 기준으로 활용하시기 바랍니다. ESG는 Environmental(환경), Social(사회), Governance(거버넌스)의 세 가지 측면으로 구성됩니다. 환경 측면은 기업이 환경 친화적으로 운영하고자 하는 의지를 나타냅니다. 이는 친환경 제품 제조, 에너지 효율성 향상 등을 포함합니다. 사회적 측면은 기업이 사회적 책임을 다하는 것을 의미하며, 노동자 권리 보호, 다양성과 포용성 강화 등이 포함됩니다. 거버넌스 측면은 기업 내부의 투명성과 윤리성을 강화하여 이해관계자들의 신뢰를 유지하는 것을 중요시합니다. 이는 감사와 감사위원회의 설정, 보상 정책의 공정성 등을 포함합니다. 이러한 ESG 요소들은 기업의 장기적인 지속 가능성과 가치 창출에 영향을 미치며, 적절한 관리를 통해 사회적으로 책임 있는 기업 경영을 추구합니다. 그 밖에도 유사한 관점에서 당신이 알고 있는 관련 정보나 기준에 근거하여 평가하셔도 괜찮습니다. 이제부터, 주어지는 문장을 ESG관점에서 E, S, G를 각 섹터별로 평가할 것입니다. 해당 ESG요소 관점에서 각 문장이 긍정적인 내용일 경우 1, 관련 없는 내용이거나 중립적 내용의 경우 0, 부정적인 내용의 경우 -1로 분류하세요. 결과는 무조건 리스트 형태로, 예를 들어 (E평가값,S평가값,G평가값) 으로만 출력해야 합니다. 절대  문장으로 출력하지 마십시오. 예를 들어, E에 관련 없음, S 관점에서 부정적인 내용, G관점에서 긍정적인 내용인 경우, 반환값은 (0, -1, 1) 로 출력합니다. 이제 평가를 시작하십시오."},
            {"role": "user", "content": sentence}
        ]
    )

    result = response['choices'][0]['message']['content']
    if "1" in result:
        return 1
    elif "0" in result:
        return 0
    elif "-1" in result:
        return -1
    return result  # 기본값


def calculate_document_esg_score(sentences):
    """문서의 ESG 친화도 총점을 계산합니다."""
    score = 0
    for sentence in sentences:
        score += evaluate_esg(sentence)  # 점수를 더합니다.
    return score


In [ ]:
#Text file path setting in # Google Drive (Example: "/Content/Drive/My Drive/Your_file.txt")
file_path = "/content/drive/MyDrive/document1.txt"
text = load_text_from_file(file_path)
sentences = split_into_sentences(text)

# Evaluate the ESG affinity of each sentence and save the results.
results = []
for sentence in sentences:
    esg_score = evaluate_esg(sentence)
    results.append({'Sentence': sentence, 'ESG Score': esg_score})

# Create the result as a data frame.
df_results = pd.DataFrame(results)

# Print the data frame.
print(df_results)

In [ ]:
# It is a code assumed for comparison with the data with the data with the E, S, and G scores.
# In fact, the API call method for E, S, and G scores may vary depending on whether Openai supports.

import pandas as pd

# ESG score data generated by the program
# Home: E, S, and G scores for each document are recorded in separate columns.
generated_esg_scores = pd.read_csv('/content/drive/MyDrive/졸업전시회/esg_combined.csv')

# ESG score data loaded by people
# Household: The file path is '/content/drive/my drive/Human_esg_scores_detailed.csv'
human_esg_scores = pd.read_csv('/content/drive/MyDrive/졸업전시회/merged_df.csv')

# Merge two data frames for comparison.
comparison_df = pd.merge(generated_esg_scores, human_esg_scores, on='Column1', suffixes=('_generated', '_human'))

# E, S, G adds to each element to new columns.
comparison_df['E_Match'] = comparison_df['E_generated'] == comparison_df['E_human']
comparison_df['S_Match'] = comparison_df['S_generated'] == comparison_df['S_human']
comparison_df['G_Match'] = comparison_df['G_generated'] == comparison_df['G_human']

# Count the number of documents that all elements match.
all_match_count = comparison_df[comparison_df[['E_Match', 'S_Match', 'G_Match']].all(axis=1)].shape[0]

# Print the result.
print(comparison_df)
print(f"Number of documents where all ESG scores match: {all_match_count}")


                                               Column1  E_generated  \
0                               인공지능이 일상의 서비스로 파고들고 있다            0   
1     8·15 특별사면 대상에 포함, 복권된 이재용 삼성전자 부회장이 본격적으로 책임 ...            0   
2     ① 윤송이 엔씨소프트 최고전략책임자 윤송이 CSO는 2011년 엔씨소프트에 AI ...            0   
3                             다양성과 경쟁력 카타르 월드컵이 막을 내렸다            0   
4     동효정 기자  삼성전자가 스웨덴과 노르웨이에 뉴스룸을 개설하고 북유럽 시장 공략을...            0   
..                                                 ...          ...   
651  확정된 2023년도 평가지표와 관련한 세부 내용은 내년 1월 2023년도 평가대상 ...            0   
652                      회사별로 대외 기부금 출연을 위한 승인 절차를 거쳤다            0   
653  회장 승진할까지배구조 향방은현재 삼성의 지배구조는 ‘이재용 삼성전자 부회장 등 오너...            0   
654        희귀병을 앓는 김민종군의 가족은 최근 삼성전자 임직원들에게 직접 손편지를 썼다            0   
655              희망디딤돌은 자립을 준비하는 청소년에게 주거를 제공하는 프로그램이다            0   

     S_generated  G_generated  E_human  S_human  G_human  E_Match  S_Match  \
0            0.0            0        0        0        0     True    

In [ ]:
import pandas as pd
import json
from tqdm import tqdm

# Json file path
file_path = '/content/drive/MyDrive/SE22_ESG original.json'

# Import file
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extracting sentence and ESG evaluation
results = []
for year, categories in tqdm(data.items(), desc="Years"):
    for category, sentences in tqdm(categories.items(), desc="Categories", leave=False):
        for sentence in tqdm(sentences, desc="Sentences", leave=False):
# Function calls for evaluating ESG affinity for each sentence (virtual)
# In this case, set all scores to (0, 0, 0) as an example
            esg_score = evaluate_esg(sentence)
            results.append({'Year': year, 'Category': category, 'Sentence': sentence, 'ESG Score': esg_score})

# Create the result as a data frame.
df_results = pd.DataFrame(results)

# Data frame output
print(df_results.head())
